By Akinde Kadjo

**Goal:** The goal of this project is to distinguish normal electrocardiogram (ECG) heartbeat signals from abnormal ones. The Dataset was obtained from [Kaggle](https://www.kaggle.com/datasets/shayanfazeli/heartbeat/data). Its original source is [The PTB Diagnostic ECG Database](https://www.physionet.org/content/ptbdb/1.0.0/).